# LOCALIZACIONES DEFINIDAS
Posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
Importar librerias 

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer

Tokens

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

Llamar a la api

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

# Londres

## Funciones de llamada a la API


In [10]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response 

In [11]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        
    response2 = [name , type_, lat, lng]

    return response2

In [12]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Longitude", 3:"Latitude"})
    return df1

In [13]:
def df_final(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    return c

In [23]:
a = llamada_API(url, singapore_loc, "vegan")

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '56aa6e34498ed42196c9caf0',
   'name': 'nomVnom',
   'contact': {},
   'location': {'address': '#03-105/106/107 Clarke Quay Central',
    'crossStreet': '6 Eu Tong Sen Street',
    'lat': 1.2889771012307392,
    'lng': 103.84693000137969,
    'labeledLatLngs': [{'label': 'display',
      'lat': 1.2889771012307392,
      'lng': 103.84693000137969}],
    'distance': 357,
    'postalCode': '059817',
    'cc': 'SG',
    'neighborhood': 'Singapore River',
    'city': 'Singapore',
    'country': 'Singapore',
    'formattedAddress': ['#03-105/106/107 Clarke Quay Central (6 Eu Tong Sen Street)',
     '059817',
     'Singapore']},
   'categories': [{'id': '4bf58dd8d48988d1d3941735',
     'name': 'Vegetarian / Vegan Restaurant',
     'pluralName': 'Vegetarian / Vegan Restaurants',
     'shortName': 'Vegetarian / Vegan',
     'icon':

In [27]:
b = escoger_datos(a)

In [28]:
crear_df(b)

,Name,Type,Longitude,Latitude
0,nomVnom,Vegetarian / Vegan Restaurant,1.288977,103.84693
1,Fill a Pita,Vegetarian / Vegan Restaurant,1.284031,103.847624
2,Annalakshmi,Vegetarian / Vegan Restaurant,1.288414,103.843472
3,Eight Treasures Vegetarian 八宝素食馆,Vegetarian / Vegan Restaurant,1.281538,103.844667
4,Ci Yan Organic Vegetarian Restaurant,Vegetarian / Vegan Restaurant,1.282014,103.844734
5,Lotus Kitchen,Vegetarian / Vegan Restaurant,1.284966,103.844385
6,Kitchen by Food Rebel,Vegetarian / Vegan Restaurant,1.280879,103.848334
7,Well Dressed Salad Bar & Café,Vegetarian / Vegan Restaurant,1.281462,103.844736
8,SaladStop!,Salad Place,1.285575,103.84732
9,Sarnies,Sandwich Place,1.281556,103.848087


geometry --> mongo --> 2dsphere --> geoqueries

# Londres

In [14]:
df_Nursery_school = df_final(url, san_francisco_loc, "Nursery School")

In [15]:
df_Elementary_school = df_final(url, san_francisco_loc, "Elementary School")

In [16]:
df_Middle_school = df_final(url, san_francisco_loc, "Middle School")

In [17]:
df_Nursery_school

,Name,Type,Longitude,Latitude
0,Kids by the Bay - Financial District,Daycare,37.785471,-122.398037
1,Japanese YWCA Building / Nihonmachi Little Fri...,Nursery School,37.787017,-122.430609
2,C5 Children's School,Nursery School,37.778864,-122.414584
3,Tiny Giants Too!,Nursery School,37.797872,-122.435021
4,Kids by the Bay - Civic Center,Preschool,37.77943,-122.411819
5,Judith Baker CD,Nursery School,37.776839,-122.411802
6,Morning Glory Nursery School,Nursery School,37.776355,-122.443699


In [19]:
df_Middle_school

,Name,Type,Longitude,Latitude
0,Francisco Middle School,Middle School,37.804937,-122.410958
1,Fusion Academy San Francisco,Private School,37.801833,-122.40142
2,Bessie Carmichael Middle School/Filipino Educa...,Middle School,37.780628,-122.400532
3,CAIS Middle School Campus,Middle School,37.78173,-122.423539
4,Schools of the Sacred Heart,High School,37.794496,-122.433985
5,Marina Middle School,Middle School,37.801905,-122.435375
6,Town School for Boys,Middle School,37.792041,-122.440143
7,Chinese American International School,Middle School,37.775275,-122.422111
8,Gateway Middle School,Middle School,37.779429,-122.435971
9,Creative Arts Charter School,Middle School,37.779772,-122.436125


In [20]:
df_Elementary_school

,Name,Type,Longitude,Latitude
0,Jean Parker Elementary,Elementary School,37.797641,-122.411043
1,Gordon J Lau Elementary,Elementary School,37.794044,-122.409019
2,Garfield Elementary School,Elementary School,37.802044,-122.406822
3,Edwin and Anita Lee Newcomer School,Elementary School,37.794781,-122.404373
4,John Yehall Chin Elementary School,Elementary School,37.798412,-122.403211
5,Chinese Education Center Elementary School,Elementary School,37.794757,-122.404204
6,Yick Wo Elementary School,Elementary School,37.801789,-122.416288
7,Nam Kue Chinese School,Elementary School,37.793196,-122.405458
8,Cathedral School for Boys,Elementary School,37.792128,-122.414404
9,Ecole Notre Dame des Victoires,Elementary School,37.791159,-122.406354
